In [2]:
%pip install pandas seaborn matplotlib numpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 218.6 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.7/12.7 MB 696.2 kB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 733.7 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 1.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.2/309.2 kB 631.5 kB/s eta 0:00:000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 1.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 1.3 MB/s eta 0:00:0000:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.2/103.2 kB 295.7 kB/s eta 0:00:000:01:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 505.5/505.5 kB 1.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.4/345.4 kB 1.1 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [7]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import os

In [9]:

# Data Loading

current_dir = os.getcwd()
data_folder = os.path.join(current_dir, '../data')

# Load the data from the CSV file
benin_data = pd.read_csv(os.path.join(data_folder, 'benin-malanville.csv'))
sierra_leone_data = pd.read_csv(os.path.join(data_folder, 'sierraleone-bumbuna.csv'))
togo_data  = pd.read_csv(os.path.join(data_folder, 'togo-dapaong_qc.csv'))

In [10]:
# data cleaning before analysis
def clean_data(data):

    # This function performs data cleaning on the input DataFrame:
    # 1. Replaces inf and -inf values with NaN
    # 2. Converts the 'Timestamp' column to datetime format, handling conversion errors
    # 3. Fills missing values in numeric columns with the column mean
    # 4. Removes any duplicate rows
    # 5. Drops columns that have NaN values for all rows

    data = data.replace([np.inf, -np.inf], np.nan)
    data["Timestamp"] = pd.to_datetime(data["Timestamp"], errors="coerce")

    numeric_columns = data.select_dtypes(include=[np.number]).columns
    data[numeric_columns] = data[numeric_columns].apply(lambda x: x.fillna(x.mean()))

    data.drop_duplicates(inplace=True)
    
    null_columns = data.cloumns[data.isnull().sum() == len(data)]
    data.drop(columns = null_columns, inplace=True)

    return data


In [ ]:
def perform_EDA(data, location_name):
    data = clean_data(data)

    if(data.empty):
        print(f"No Available data after cleaning for {location_name}")
        return 

    # perform eda analysis   
    data.set_index('Timestamp', inplace=True)

    # 1. Summary Statistics
    summary_stats = data.describe()
    print(f"\nSummary statistics for {location_name}:")
    print(summary_stats)

    # 2. Data Quality Check
    missing_values = data.isnull().sum()
    print(f"\nMissing values for {location_name}:")
    print(missing_values)

    # 3. Time Series Analysis
    variables = ['GHI', 'DNI', 'DHI', 'Tamb']
    plt.figure(figsize=(12, 6))
    sns.lineplot(data=data[variables])
    plt.title(f"Time-series trends for {location_name}")
    plt.xlabel("Time")
    plt.ylabel("Value")
    plt.show()

    # 4. Correlation Analysis
    correlation_matrix = data.corr()
    plt.figure(figsize=(12, 8))
    sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f")
    plt.title(f"Correlation Matrix for {location_name}")
    plt.show()

    # 5. Wind Analysis
    wind_variables = ['WS', 'WSgust', 'WSstdev', 'WD', 'WDstdev']
    plt.figure(figsize=(12, 6))
    sns.lineplot(data=data[wind_variables])
    plt.title(f"Wind Speed and Direction for {location_name}")
    plt.xlabel("Time")
    plt.ylabel("Value")
    plt.show()

    # 6. Temperature Analysis
    temp_variables = ['TModA', 'TModB', 'Tamb']
    plt.figure(figsize=(12, 6))
    sns.lineplot(data=data[temp_variables])
    plt.title(f"Module and Ambient Temperatures for {location_name}")
    plt.xlabel("Time")
    plt.ylabel("Temperature (°C)")
    plt.show()

    # 7. Histograms
    histogram_variables = ['GHI', 'DNI', 'DHI', 'WS', 'Tamb']
    for var in histogram_variables:
        plt.figure(figsize=(12, 6))
        sns.histplot(data[var], kde=True)
        plt.title(f"Histogram of {var} for {location_name}")
        plt.xlabel(var)
        plt.ylabel("Count")
        plt.show()

    # 8. Box Plots
    plt.figure(figsize=(12, 6))
    sns.boxplot(data=data[variables])
    plt.title(f"Box Plot for {location_name}")
    plt.show()

    # 9. Scatter Plots
    scatter_pairs = [('GHI', 'Tamb'), ('WS', 'WSgust')]
    for x, y in scatter_pairs:
        plt.figure(figsize=(12, 6))
        sns.scatterplot(data=data, x=x, y=y, hue='Precipitation', palette='viridis', s=20)
        plt.title(f"Scatter Plot of {x} vs {y} for {location_name}")
        plt.xlabel(x)
        plt.ylabel(y)
        plt.show()
      